In [1]:
from Utils.loadset import getDataSet
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import tensorflow_probability as tfp
import os
physical_devices = tf.config.list_physical_devices('GPU') 
print("Num GPUs:", len(physical_devices)) 

from trainer import Trainer
try:
    from Utils.connection_cfg import *
except Exception as e:
    PSWD = None
    USRN = None
    
from Utils.Data import dataWrapper
from Utils.transform import cutOut

Num GPUs: 1
Num GPUs Available: 1


Using TensorFlow backend.


In [2]:
#slices = [272,304,272,304]
slices = [256,320,256,320]
cutOutFrame = cutOut(slices)

PRETRAINING_TRANSFORMATIONS = [cutOutFrame]
DatasetFolder = "./Data/RAW"
PathToData = os.path.join(DatasetFolder,"MonthPNGData")
dimension = (64,64)
channels = 5
optimizer = Adam( lr = 1e-3 )
batch_size=10
def provideData(flatten=False,dimension=dimension,batch_size=10,transform=None,preTransformation=PRETRAINING_TRANSFORMATIONS):

    getDataSet(DatasetFolder,year=[2017],username=USRN,pswd=PSWD)
    train,test = dataWrapper(PathToData,
                            dimension=dimension,
                            channels=channels,
                            batch_size=batch_size,
                            overwritecsv=True,
                            flatten=flatten,
                            onlyUseYears=[2017],
                            transform=transform,
                            preTransformation=preTransformation)
    
    return train,test



tfd = tfp.distributions
def NLL(y_true, y_hat):
    return -y_hat.log_prob(y_true)

train, test = provideData(flatten=False,dimension=dimension,batch_size=batch_size,transform=None,preTransformation=PRETRAINING_TRANSFORMATIONS)

Found Year :  2017 => won't download this year again... please check for consistency
Finished Loading Dataset
 


In [11]:


tfd = tfp.distributions
def NLL(y_true, y_hat):
    return -y_hat.log_prob(y_true[:,:])
                           
def mixd(output):
    rate = tf.math.exp(output[:,:,:,0:1]) #A 
    s = tf.math.sigmoid(output[:,:,:,1:2])
    s = Flatten(s)
    rate= Flatten(rate)
    
    components = [tfd.Deterministic(loc=tf.zeros_like(rate)), #E
     tfd.Poisson(rate=rate) #F 
     ]
    mixture = tfd.Mixture(
          cat=tfd.Categorical(probs=[tf.concat([1-s, s],axis=3)]),#D
          components=components)
    
    return mixture
    
from tensorflow_probability.python.internal import distribution_util as dist_util
def zeroInflatedPoisson(output,event_shape=()):
    
    """    
    rate = output[:,:,:,0] #A 
    s = tf.math.sigmoid(output[:,:,:,1:2])
    
    
    event_shape = dist_util.expand_to_vector(tf.convert_to_tensor(
              tf.shape(rate[0,:,:]), name='event_shape', dtype_hint=tf.int32),tensor_name='event_shape')
    
    pos = tfp.distributions.Independent(tfp.distributions.Poisson(rate=rate),
                                        reinterpreted_batch_ndims=tf.size(event_shape))
    det = tfp.distributions.Independent(
                            tfp.distributions.Deterministic(loc=tf.zeros_like(rate)),
                            reinterpreted_batch_ndims=tf.size(event_shape))
    probs = tf.concat([1-s, s],axis=-1)
    print(probs)
    cat = tfd.Categorical(probs=probs)
    event_shape = dist_util.expand_to_vector(tf.convert_to_tensor(
              tf.shape(probs[0,:,:]), name='event_shape', dtype_hint=tf.int32),tensor_name='event_shape')
    cat2 = tfp.distributions.Independent(tfd.Categorical(probs=probs),reinterpreted_batch_ndims=tf.size(event_shape),name="ZeroInflated")
    print("CAT",cat2.event_shape)
    #tfp.distributions.Independent(mixture,reinterpreted_batch_ndims=tf.size(event_shape),name="ZeroInflated")
    #s = Flatten()(s)
    #rate= Flatten()(rate)
    #pos = tfp.distributions.Poisson(rate=rate)
    #det =  tfp.distributions.Deterministic(loc=tf.zeros_like(rate))
    print(pos)
    print(det)
    print(cat)
          
    components = [det,pos]
    mixture = tfd.Mixture(
          cat=cat,
          components=components)
    #ind = tfp.distributions.Independent(mixture,reinterpreted_batch_ndims=1,name="ZeroInflated")
    
    
    ind = tfp.distributions.Independent(mixture,reinterpreted_batch_ndims=tf.size(event_shape),name="ZeroInflated")
    print(ind,event_shape)
    return ind
    
    
    rate = Flatten()(output[:,:,:,0])
    s = Flatten()(output[:,:,:,1])
    rate = tf.math.exp(rate)
    s = tf.math.sigmoid(s)
    
    event_shape = dist_util.expand_to_vector(tf.convert_to_tensor(
              tf.shape(rate[0,:]), name='event_shape', dtype_hint=tf.int32),tensor_name='event_shape')
    
    pos = tfp.distributions.Independent(tfp.distributions.Poisson(rate=rate),
                                        reinterpreted_batch_ndims=tf.size(event_shape))
    det = tfp.distributions.Independent(
                            tfp.distributions.Deterministic(loc=tf.zeros_like(rate)),
                            reinterpreted_batch_ndims=tf.size(event_shape))
    #s = tf.expand_dims(s,-1)
    c_event_shape = dist_util.expand_to_vector(tf.convert_to_tensor(
              tf.shape(tf.concat([1-s,s],axis=1)), name='event_shape', dtype_hint=tf.int32),tensor_name='event_shape')
    print(tf.concat([1-s,s],axis=1),c_event_shape)
    #print(tf.concat([1-s, s],axis=1))
    cat = tfd.Categorical(probs=tf.concat([1-s, s],axis=1))
    components = [det,pos]
    print(components)
    print(cat)
    
    mixture = tfd.Mixture(cat=cat,
          components=components)
    ind = tfp.distributions.Independent(mixture,reinterpreted_batch_ndims=tf.size(64,64,1),name="ZeroInflated")
    return ind
    """
    s = Flatten()(output[:,:,:,:1])
    rate= Flatten()(output[:,:,:,1:2])
    rate = tf.math.exp(rate) #A 
    s = tf.math.sigmoid(s)
    
    logits = tf.stack([1-s, s],axis=2)
    cat = tfd.Categorical(logits=logits)

    
    components = [tfd.Deterministic(loc=tf.zeros_like(rate)),
                  tfd.Poisson(rate=rate)]

    mixture = tfd.Mixture(
          cat=cat,
          components=components)
    mixture = tfp.distributions.Independent(mixture,name="ZeroInflated")
    return mixture

def testnetZeroInflatedPoisson(input_shape,
           n_predictions=1,
           simpleclassification=None,
           flatten_output=False,
           activation_hidden="relu",
           activation_output="relu"):


    

    inputs = Input(shape=input_shape) 

    conv01 = Conv2D(10, kernel_size=(3, 3), padding="same")(inputs)       # 10 x 64x64
    conv01 = Activation(activation_hidden)(conv01)
    conv01_pool = MaxPooling2D((2, 2), strides=(2, 2))(conv01)            # 10 x 32x32


    conv02 = Conv2D(20, kernel_size=(3, 3), padding="same")(conv01_pool)  # 20 x 32x32
    conv02 = Activation(activation_hidden)(conv02)
    conv02_pool = MaxPooling2D((2, 2), strides=(2, 2))(conv02)            # 20 x 16x16


    conv03 = Conv2D(20, kernel_size=(3, 3), padding="same")(conv02_pool)  # 20 x 16x16
    conv03 = Activation(activation_hidden)(conv03)
    conv03_pool = MaxPooling2D((2, 2), strides=(2, 2))(conv03)            # 20 x 8x8


    conv04 = Conv2D(20, kernel_size=(3, 3), padding="same")(conv03_pool)  # 20 x 8x8
    conv04 = Activation(activation_hidden)(conv04)
    conv04_pool = MaxPooling2D((2, 2), strides=(2, 2))(conv04)            # 20 x 4x4


    ### UPSAMPLING:
    up04 = UpSampling2D((2, 2))(conv04_pool)    # 20 x 8x8
    up04 = concatenate([conv04, up04], axis=3)  # 20+20 x 8x8


    up03 = UpSampling2D((2, 2))(up04)           # 40 x 16x16
    up03 = concatenate([conv03, up03], axis=3)  # 20+40 x 16x16


    up02 = UpSampling2D((2, 2))(up03)           # 60 x 32x32
    up02 = concatenate([conv02, up02], axis=3)  # 20+60 x 32x32


    up01 = UpSampling2D((2, 2))(up02)           # 80 x 64x64
    up01 = concatenate([conv01, up01], axis=3)  # 10+80 x 64x64

    
    layer = Conv2D(2, (1, 1), activation="linear")(up01)  # 1 x 64x64
    #output = Flatten()(output)
    
    #output = Dense(4096,activation="linear",use_bias=False)(output)
    #output = Activation(tf.math.softplus)(output)
    
    layer_1 = Flatten()(layer[:,:,:,:1])
    layer_2 = Flatten()(layer[:,:,:,1:2])
    layer_1 = Dense(64*64,activation="tanh")(layer_1)
    layer_2 = Dense(64*64,activation="tanh")(layer_2)
    
    layer_stacked = tf.stack([layer_1,layer_2],axis=-1)
    layer = tf.reshape(layer_stacked, tf.shape(layer))
    output = tfp.layers.DistributionLambda(zeroInflatedPoisson,name="ZPoisson")(layer)
    
    
    #output = tfp.layers.IndependentPoisson((64,64,1))(output)
    model = Model(inputs=inputs, outputs=output)
    return model


print(tfp.layers.DistributionLambda(zeroInflatedPoisson,name="ZPoisson"))

In [12]:
model = testnetZeroInflatedPoisson((*dimension,5))
model.compile(loss=lambda y, model_normal: -model_normal.log_prob(y), optimizer=Adam( lr= 1e-3 ),metrics = ["mse","mae"])
#model.compile(loss="mse", optimizer=Adam( lr= 1e-4 ))
model.summary()

logits:  Tensor("ZPoisson_1/stack:0", shape=(None, 4096, 2), dtype=float32)
s     : Tensor("ZPoisson_1/Sigmoid:0", shape=(None, 4096), dtype=float32)
rate  : Tensor("ZPoisson_1/Exp:0", shape=(None, 4096), dtype=float32)
tfp.distributions.Categorical("ZPoisson_1_Categorical", batch_shape=[?, 4096], event_shape=[], dtype=int32)
[<tfp.distributions.Deterministic 'ZPoisson_1_Deterministic' batch_shape=[?, 4096] event_shape=[] dtype=float32>, <tfp.distributions.Poisson 'ZPoisson_1_Poisson' batch_shape=[?, 4096] event_shape=[] dtype=float32>]
tfp.distributions.Independent("ZPoisson_1_ZeroInflated", batch_shape=[?], event_shape=[4096], dtype=float32)
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 64, 64, 5)]  0                                            
__________________________________

In [ ]:
history = model.fit(train,validation_data=test,epochs=30)

Epoch 1/30
logits:  Tensor("model_1/ZPoisson/stack:0", shape=(None, 4096, 2), dtype=float32)
s     : Tensor("model_1/ZPoisson/Sigmoid:0", shape=(None, 4096), dtype=float32)
rate  : Tensor("model_1/ZPoisson/Exp:0", shape=(None, 4096), dtype=float32)
tfp.distributions.Categorical("model_1_ZPoisson_Categorical", batch_shape=[?, 4096], event_shape=[], dtype=int32)
[<tfp.distributions.Deterministic 'model_1_ZPoisson_Deterministic' batch_shape=[?, 4096] event_shape=[] dtype=float32>, <tfp.distributions.Poisson 'model_1_ZPoisson_Poisson' batch_shape=[?, 4096] event_shape=[] dtype=float32>]
tfp.distributions.Independent("model_1_ZPoisson_ZeroInflated", batch_shape=[?], event_shape=[4096], dtype=float32)
logits:  Tensor("model_1/ZPoisson/stack:0", shape=(None, 4096, 2), dtype=float32)
s     : Tensor("model_1/ZPoisson/Sigmoid:0", shape=(None, 4096), dtype=float32)
rate  : Tensor("model_1/ZPoisson/Exp:0", shape=(None, 4096), dtype=float32)
tfp.distributions.Categorical("model_1_ZPoisson_Categoric

In [ ]:

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.pyplot import figure
import numpy as np
%matplotlib inline
model = t.model
pred = None
label = None
pred = 10
test.on_epoch_end()
mean = []
stdev = []
label = []
for x,y in test:
    for i in range(batch_size):
        if y[i,:,:,:].max() > 0:
            pred = model(np.array([x[i,:,:,:]]))
            mean.append(pred.mean())
            stdev.append(pred.stddev())
            label.append(y[i,:,:,:])
            if len(label) == 10:
                break
        #mean = model(x).mean()
        #std = model(x).stddev()
        #label = y
        break
    if len(label) == 10:
                break

In [ ]:
fig, axes = plt.subplots(10, 3, figsize=(16,8*10),dpi=64)
#fig.set_title(["mean","stdev","label"])
for batch,img in enumerate(mean):
    if batch == 10:
        break
    axes[batch,0].imshow(img[0,:,:,0],cmap="gray")
    axes[batch,1].imshow(stdev[batch][0,:,:,0],cmap="gray")
    axes[batch,2].imshow(label[batch][:,:,0],cmap="gray")